<a href="https://colab.research.google.com/github/rjac-ml/Gender-Bias-Wiki/blob/add-colab-code/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!rm -r sample_data/

rm: cannot remove 'sample_data/': No such file or directory


In [210]:
from psutil import virtual_memory
import os 

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

NCPU = os.cpu_count()

print("Number  of CPU {}".format(NCPU))

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!
Number  of CPU 4


### Libraries

In [54]:
%%capture
!pip install datasets transformers spacy

In [55]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import pipeline
import spacy
import re

In [56]:
nlp = spacy.load("en_core_web_sm")

### Data Sourcing

Clean articles from Wikipedia https://huggingface.co/datasets/wikipedia 

In [57]:
df = load_dataset("wikipedia", "20220301.en")

Reusing dataset wikipedia (/root/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

### Data Understanding

In [58]:
df

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})

In [59]:
df.get("train")["title"][17]

'Allan Dwan'

In [60]:
text = df.get("train")[17].get("text")

In [61]:
text

'Allan Dwan (born Joseph Aloysius Dwan; April 3, 1885 – December 28, 1981) was a pioneering Canadian-born American motion picture director, producer, and screenwriter.\n\nEarly life\nBorn Joseph Aloysius Dwan in Toronto, Ontario, Canada, Dwan, was the younger son of commercial traveler of woolen clothing Joseph Michael Dwan (1857–1917) and his wife Mary Jane Dwan, née Hunt. The family moved to the United States when he was seven years old on December 4, 1892 by ferry from Windsor to Detroit, according to his naturalization petition of August 1939. His elder brother, Leo Garnet Dwan (1883–1964), became a physician.\n\nAllan Dwan studied engineering at the University of Notre Dame and then worked for a lighting company in Chicago. He had a strong interest in the fledgling motion picture industry, and when Essanay Studios offered him the opportunity to become a scriptwriter, he took the job. At that time, some of the East Coast movie makers began to spend winters in California where the c

In [62]:
text

'Allan Dwan (born Joseph Aloysius Dwan; April 3, 1885 – December 28, 1981) was a pioneering Canadian-born American motion picture director, producer, and screenwriter.\n\nEarly life\nBorn Joseph Aloysius Dwan in Toronto, Ontario, Canada, Dwan, was the younger son of commercial traveler of woolen clothing Joseph Michael Dwan (1857–1917) and his wife Mary Jane Dwan, née Hunt. The family moved to the United States when he was seven years old on December 4, 1892 by ferry from Windsor to Detroit, according to his naturalization petition of August 1939. His elder brother, Leo Garnet Dwan (1883–1964), became a physician.\n\nAllan Dwan studied engineering at the University of Notre Dame and then worked for a lighting company in Chicago. He had a strong interest in the fledgling motion picture industry, and when Essanay Studios offered him the opportunity to become a scriptwriter, he took the job. At that time, some of the East Coast movie makers began to spend winters in California where the c

### Data Selection

In [65]:
sdf = df.get("train").select([0,3,5])

In [66]:
sdf = sdf.map(lambda line: {"bio_flag": len(re.findall(r"\b(biography|early life)\b",line["text"].lower()))})

  0%|          | 0/3 [00:00<?, ?ex/s]

In [67]:
df = df.map(lambda line: {"bio_flag": len(re.findall(r"\b(biography|early life)\b",line["text"].lower()))})

  0%|          | 0/6458670 [00:00<?, ?ex/s]

In [68]:
bios = df.get("train").filter(lambda line: line['bio_flag']>0)

  0%|          | 0/6459 [00:00<?, ?ba/s]

In [69]:
bios.save_to_disk("/content/drive/MyDrive/pipeline/wiki")

Flattening the indices:   0%|          | 0/573 [00:00<?, ?ba/s]

In [78]:
sdf = bios.select([0,3,5])

In [194]:
def sentences(line):
    chunks = []
    for l in line["text"]:
        chunks += [l for l in l.split("\n\n") if len(l) > 30]
    return {'chunks': chunks}

In [159]:
bios = bios.map(lambda line: {"intro":line["text"].split("\n\n")[0]} )

  0%|          | 0/572797 [00:00<?, ?ex/s]

In [182]:
bios = bios.map(lambda lines:{"PERSON": [np.array([e.label_ == "PERSON" for e in nlp(i).ents]).any() for i in lines["intro"]] },batched=True,num_proc=4)

#0:   0%|          | 0/144 [00:00<?, ?ba/s]

#1:   0%|          | 0/144 [00:00<?, ?ba/s]

#2:   0%|          | 0/144 [00:00<?, ?ba/s]

#3:   0%|          | 0/144 [00:00<?, ?ba/s]

In [183]:
bios.save_to_disk("/content/drive/MyDrive/pipeline/wiki")

In [186]:
bios = bios.filter(lambda line : line["PERSON"] == True)

  0%|          | 0/573 [00:00<?, ?ba/s]

In [189]:
lines = bios.map(sentences,batched=True,remove_columns=bios.column_names)

  0%|          | 0/531 [00:00<?, ?ba/s]

In [190]:
lines

Dataset({
    features: ['chunks'],
    num_rows: 6922383
})

In [191]:
lines.save_to_disk("/content/drive/MyDrive/pipeline/bio")

In [196]:
lines[125127:125129]

{'chunks': ['With the financial backing of several Jewish investors, including one of the owners, Abraham Bankier, Schindler signed an informal lease agreement on the factory on 13 November 1939 and formalised the arrangement on 15 January 1940. He renamed it Deutsche Emailwarenfabrik (German Enamelware Factory) or DEF, and it soon became known by the nickname "Emalia". He initially acquired a staff of seven Jewish workers (including Abraham Bankier, who helped him manage the company) and 250 non-Jewish Poles. At its peak in 1944, the business employed around 1,750 workers, a thousand of whom were Jews. Schindler also helped run Schlomo Wiener Ltd, a wholesale outfit that sold his enamelware, and was leaseholder of Prokosziner Glashütte, a glass factory.',
  "Schindler's ties with the Abwehr and his connections in the Wehrmacht and its Armaments Inspectorate enabled him to obtain contracts to produce enamel cookware for the military. These connections also later helped him protect his 

In [206]:
lines = lines.map(lambda line: {"F":len(re.findall(r"\b(she|her|hers|herself)\b",line.get("chunks").lower())),"M":len(re.findall(r"\b(he|his|him|himself)\b",line.get("chunks").lower()))},num_proc=4)

#0:   0%|          | 0/1730596 [00:00<?, ?ex/s]

#1:   0%|          | 0/1730596 [00:00<?, ?ex/s]

#3:   0%|          | 0/1730595 [00:00<?, ?ex/s]

#2:   0%|          | 0/1730596 [00:00<?, ?ex/s]

In [207]:
lines[30004:30010]

{'F': [7, 4, 0, 4, 2, 3],
 'M': [1, 3, 0, 0, 0, 1],
 'chunks': ['Although Czolgosz repeatedly denied Goldman\'s involvement, the police held her in close custody, subjecting her to what she called the "third degree". She explained her housemates\' distrust of Czolgosz, and the police finally recognized that she had not had any significant contact with the attacker. No evidence was found linking Goldman to the attack, and she was released after two weeks of detention. Before McKinley died, Goldman offered to provide nursing care, referring to him as "merely a human being". Czolgosz, despite considerable evidence of mental illness, was convicted of murder and executed.',
  'Throughout her detention and after her release, Goldman steadfastly refused to condemn Czolgosz\'s actions, standing virtually alone in doing so. Friends and supporters—including Berkman—urged her to quit his cause. But Goldman defended Czolgosz as a "supersensitive being" and chastised other anarchists for abandoning

In [209]:
lines.save_to_disk("/content/drive/MyDrive/pipeline/bio")

### Data Modeling

Deep Learning and Machine learning algorithms are develop to find patters and distribution in the data. This mean that if there is no Bias toward the Language use in a text the model would not be able to find a patter is we degendered the text. As the paper mention  "*In an ideal world, we would expect little difference between texts describing men, women, and people with other gender identities, aside from the use of explicitly gendered words, like pronouns or names. A machine learning model, then, would be unable to pick up on statistical differences among gender labels (i.e., gender bias), because such differences would not exist*" 